In [1]:
%reload_ext setup

INFO:setup:IPython environment configured successfully!


## Introduction to Agents

In [2]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt="You are a helpful assistant",
    critic_prompt="You are a critic, giving useful fedback. If happy with the result, say 'APPROVE'. If not, give feedback."
)
# display(agents)

# Cell 3: Run the chat with simplified output
message = "Hello, tell a joke?"
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):  # Only show actual text messages between agents
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):  # Show final result
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Hello, tell a joke?
matcher: Why don't skeletons fight each other? They don't have the guts!
critic: APPROVE
Task completed. Stop reason: Text 'APPROVE' mentioned


## Demo Example (1 registrants)

In [3]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agents(state_path="model_config.yaml")

# Cell 3: Run the chat with matching task
message = (
    "Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. "
    "Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), "
    "and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. "
    "Output matches in JSON format as per the matching rules."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. Output matches in JSON format as per the matching rules.
matcher: The registration with ZIP code 55409 is primarily serviced by two suppliers: "iSoalr" and "All Energy Solar", as per their service areas. I'll need to verify their capacity and match the registration based on that, while considering the matching rules for effective distribution. Let's proceed with the matching based on these conditions:

### Registration Details
- **Registration Number**: SPUS62763
- **Contact ZIP**: 55409
- **Required Optimal Panels**: 800.0

### Supplier Offer and Capacity Verification
- **Suppliers Servicing 55409**: "All Energy Solar" (does not match ZIP 55409), "iSoalr"

### Product Offers (for iSoalr)
- **Product Type**: Sola

INFO:openai._base_client:Retrying request to /chat/completions in 6.058000 seconds


critic: Upon reviewing the output provided by the <role_matching_expert>, a few points need clarification to ensure it adheres to the <matching_rules> and <distribution_rules>:

1. **ZIP code Alignment**: The registration with ZIP code 55409 can indeed be serviced by "iSoalr" as it falls within their defined service area.

2. **Supplier Capacity Verification**: Since the output confirms "iSoalr" is selected, it indicates available capacity. However, it's important to check if the "All Energy Solar" capacity is fully exhausted before finalizing the assignment, as redistribution might be required later for optimal balance.

3. **Product Needs Alignment**: The registration's optimal amount of panels (800) is significantly large, and it's unclear if this was accurately considered when matching to a panel capacity offered by "iSoalr".

4. **Distribution Check**: The distribution logic should ensure that no single supplier becomes overwhelmed, especially when multiple suppliers service overl

INFO:openai._base_client:Retrying request to /chat/completions in 21.684000 seconds


matcher: The feedback indicates several clarifications and validations to ensure the matching aligns with the <matching_rules> and <distribution_rules>. Let's address these points:

1. **ZIP Code Alignment**: Confirmed that "iSoalr" services ZIP code 55409.

2. **Supplier Capacity Verification**: Before finalizing, it's crucial to verify if "All Energy Solar" is truly excluded from servicing 55409, to check for potential capacity balance needs.

3. **Product Needs Alignment & Panel Capacity Consideration**: Let's interpret the registration's optimal panel amount (800 panels) versus available capacity ("Silfab HG" at 410). I'll re-evaluate if this alignment addresses the large capacity required.

4. **Distribution Check**: Utilize a fair distribution, according to service overlap and ensure one supplier isn't overwhelmed.

### Steps for Review:
- Check service areas for both suppliers to confirm service implications and possible redistribution if necessary.
- Address the product need: G

INFO:openai._base_client:Retrying request to /chat/completions in 15.274000 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 24.342000 seconds


matcher: In this registration, the optimal number of panels (800) likely represents a significant capacity demand, which might have been misinterpreted earlier. Let's address the feedback with updated consideration:

### Correct Matching Interpretation
#### Registration and Supplier Re-evaluation:
- **Registration ZIP Code**: 55409 is still validly serviced by **iSoalr**.
- **Demand Needs**: 800 panels is quite extensive and requires reaffirmed calculation against **iSoalr**'s current product offers.

#### Supplier Capacity and Product Availability:
- **Service Confirmations**: "iSoalr" services ZIP 55409 with enough capacity.
- **Product/Panel Offers**: As errors were noted, we need reconfirmation for fulfilling the ‘800 panels demand’ through multiple product allocations or other arrangement paths offered (i.e., by increasing units).

### Proper Solution Methodology:

**Step 1: Parallel Capacity & Priority Check:**
- Confirm supplier product handling capability for substantial demand

INFO:openai._base_client:Retrying request to /chat/completions in 26.152000 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 34.842000 seconds


matcher: Thank you for providing the supplier data. Keeping in mind the registration's specific needs and the supplier's service offerings, I will formulate an appropriate match following the guidelines provided by the <matching_rules>.

### Updated Matching Steps

#### Step 1: Service Area Validation
- Validate that ZIP code **55409** is indeed within the service area of both suppliers to effectively determine viable options.

#### Step 2: Supplier Capacity Check
- Assess each supplier's capacity:
  - **iSoalr**: Capacity is 100. It covers ZIP code 55409.
  - **All Energy Solar**: Does not explicitly cover 55409 based on previously supplied ZIP codes but is considered for redistribution if necessary.

#### Step 3: Product Capacity Analysis
- Determine if **iSoalr** can meet the registrant's need for **800 panels** with "Silfab HG" available with a capacity of 410.

#### Step 4: Distribution Rules
- Consider equitable distribution rules in instances where service areas overlap or redis

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage
import os
openai_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY")
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY environment variable set.
)
result = await openai_client.create([UserMessage(content="What is the capital of France?", source="user")])  # type: ignore
print(result)